In [ ]:
import os
import shutil
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def load_csv(csv_path, file_name):
    dataset = pd.read_csv(os.path.join(csv_path, file_name), low_memory = False)
    # dataset = pd.read_csv(os.path.join(csv_path, file_name), header = None, low_memory = False)
    dataset.fillna('1', inplace = True)
    dataset_array = np.array(dataset)
    return dataset, dataset_array

def segment(data):
    feature_set= []
    img_name = []
    demension = len(data[0])
    segment = list(range(1, demension, 5))
    for i in range(len(data)):
        for seg in segment:
            if data[i][seg] == '1':
                pass
            else:
                sub_data = data[i][seg:seg + 5]
                img = data[i][0]
                feature_set.append(sub_data)
                img_name.append(img)
    img_name = np.array(img_name)
    feature_set = np.array(feature_set)
    label = feature_set[:,4]
    w_h = feature_set[:,:4].astype('float32')
    return img_name, feature_set, label, w_h
  
def scale_bbox(cor, x_scale, y_scale):
  x, y, w ,h = cor
  x, y ,w ,h = int(np.round(x * x_scale)), int(np.round(y * y_scale)),\
               int(np.round(w * x_scale)), int(np.round(h * y_scale))
  return x, y ,w ,h

def label_transform(category):
  if category == '不良-乳汁吸附':
    label = 0
  elif category == '不良-機械傷害':
    label = 1
  elif category == '不良-炭疽病':
    label = 2
  elif category == '不良-著色不佳':
    label = 3
  else:
    label = 4
  return label

In [ ]:
# csv_path = '/content/drive/My Drive/project/mango/sample/label'
# image_path = '/content/drive/My Drive/project/mango/sample/image/'

# csv_path = '/content/drive/My Drive/project/mango/train_dev/'
# image_path = '/content/drive/My Drive/project/mango/train_dev/Train/'
# yolo_train_path = '/content/drive/My Drive/project/mango/mango_detection/yolo_train/'
# write_txt = '/content/drive/My Drive/project/mango/mango_detection/cfg/train.txt'


# csv_path = '/content/drive/My Drive/project/mango/train_dev/'
# image_path = '/content/drive/My Drive/project/mango/train_dev/Dev/
# yolo_val_path = '/content/drive/My Drive/project/mango/mango_detection/yolo_val/'
# write_txt = '/content/drive/My Drive/project/mango/mango_detection/cfg/val.txt'

csv_path = '/content/drive/My Drive/project/mango/test/'
image_path = '/content/drive/My Drive/project/mango/test/image/Test/'
yolo_test_path = '/content/drive/My Drive/project/mango/mango_detection/yolo_test/'
write_txt = '/content/drive/My Drive/project/mango/mango_detection/cfg/test.txt'

In [ ]:
# dataset, dataset_array = load_csv(csv_path, 'label.csv')
# dataset, dataset_array = load_csv(csv_path, 'train.csv')
# dataset, dataset_array = load_csv(csv_path, 'dev.csv')
dataset, dataset_array = load_csv(csv_path, 'Test_mangoXYWH.csv')

In [ ]:
len(dataset_array)

7363

In [ ]:
img_name, feature_set, label, w_h = segment(dataset_array)
x_y_min = w_h[:, 0:2].astype(np.int32)
x_y_max = np.add(w_h[:, 0:2], w_h[:, 2:]).astype(np.int32)
b_box = np.concatenate((x_y_min, x_y_max), axis = 1)

In [ ]:
data_counter = 20017

for idx in range(20017, len(img_name)):
  origin_img_path = os.path.join(image_path, img_name[idx])
  image = plt.imread(origin_img_path)
  r_image = cv2.resize(image, (416, 416))
  x_scale, y_scale = (416 / image.shape[1]), (416 / image.shape[0])
  r_image_path = os.path.join(yolo_train_path, str(data_counter) + '.jpg')
  plt.imsave(r_image_path, r_image)
  image_size = [r_image.shape[1], r_image.shape[0]]
  # save image path
  # with open(write_txt, 'a') as f:
  #   line_txt = [yolo_train_path + str(data_counter) + '.jpg', '\n']
  #   f.writelines(line_txt)
  
  data_counter += 1
  
  new_bbox = scale_bbox(b_box[idx], x_scale, y_scale)
  x, y, w, h = convert(image_size, new_bbox)
  category = label_transform(label[idx])
  with open(yolo_train_path + str(data_counter - 1) +'.txt', 'a+') as f:
    f.write(category + ' %s %s %s %s\n' % (x, y, w, h))


In [ ]:
data_counter = 0

for idx in range(len(img_name)):
  origin_img_path = os.path.join(image_path, img_name[idx])
  image = plt.imread(origin_img_path)
  r_image = cv2.resize(image, (416, 416))
  x_scale, y_scale = (416 / image.shape[1]), (416 / image.shape[0])
  r_image_path = os.path.join(yolo_train_path, str(data_counter) + '.jpg')
  plt.imsave(r_image_path, r_image)
  image_size = [r_image.shape[1], r_image.shape[0]]
  # save image path
  with open(write_txt, 'a') as f:
    line_txt = [yolo_train_path + str(data_counter) + '.jpg', '\n']
    f.writelines(line_txt)
  
  data_counter += 1
  
  new_bbox = scale_bbox(b_box[idx], x_scale, y_scale)
  x, y, w, h = convert(image_size, new_bbox)
  category = label_transform(label[idx])
  with open(yolo_train_path + str(data_counter - 1) +'.txt', 'a+') as f:
    f.write(category + ' %s %s %s %s\n' % (x, y, w, h))


In [ ]:
for i in range(0, 6280):
  old_name = os.path.join(yolo_val_path, 'val_label_' + str(i) + '.txt')
  new_name = os.path.join(yolo_val_path, str(i) + '.txt')
  os.rename(old_name, new_name)

In [ ]:
for i in range(0, 43370):
  old_name = os.path.join(yolo_train_path, 'train_label_' + str(i) + '.txt')
  new_name = os.path.join(yolo_train_path, str(i) + '.txt')
  os.rename(old_name, new_name)

#Val

In [ ]:
data_counter = 0

for idx in range(len(img_name)):
  origin_img_path = os.path.join(image_path, img_name[idx])
  image = plt.imread(origin_img_path)
  r_image = cv2.resize(image, (416, 416))
  x_scale, y_scale = (416 / image.shape[1]), (416 / image.shape[0])
  r_image_path = os.path.join(yolo_val_path, str(data_counter) + '.jpg')
  plt.imsave(r_image_path, r_image)
  image_size = [r_image.shape[1], r_image.shape[0]]
  # save image path
  # with open(write_txt, 'a') as f:
  #   line_txt = [yolo_val_path + str(data_counter) + '.jpg', '\n']
  #   f.writelines(line_txt)
  
  data_counter += 1
  # label_list = list()
  # sub_count = 1
  
  new_bbox = scale_bbox(b_box[idx], x_scale, y_scale)
  x, y, w, h = convert(image_size, new_bbox)
  category = label_transform(label[idx])
  with open(yolo_val_path + str(data_counter - 1) +'.txt', 'a+') as f:
    f.write(category + ' %s %s %s %s\n' % (x, y, w, h))


In [ ]:
data_counter

6280

#Test

In [ ]:
os.path.join(yolo_test_path, dataset_array[0, 0])

'/content/drive/My Drive/project/mango/mango_detection/yolo_test/39983.jpg'

In [ ]:
for idx in range(len(dataset_array)):
  img = plt.imread(os.path.join(image_path, dataset_array[idx, 0]))
  x, y, w, h = dataset_array[idx, 1:].astype(np.int32)
  r_image = cv2.resize(img, (416, 416))
  x_scale, y_scale = (416 / img.shape[1]), (416 / img.shape[0])
  x, y, w, h = int(np.round(x * x_scale)), int(np.round(y * y_scale)),\
               int(np.round(w * x_scale)), int(np.round(h * y_scale))
  crop_image = r_image[y:y+h, x:x+w]
  plt.imsave(os.path.join(yolo_test_path, dataset_array[idx, 0]), crop_image)
  with open(write_txt + '.txt', 'a') as f:
    line_txt = [yolo_test_path + dataset_array[idx, 0], '\n']
    f.writelines(line_txt)

In [ ]:
with open(yolo_train_path + 'train_label_' + str(0) +'.txt', 'a+') as f:
  f.write(str(1) + ' %s %s %s %s\n' % (1, 2, 3, 4))

In [ ]:
label[0]

'不良-機械傷害'

In [ ]:
counter = 0
for i in range(5):
  with open(write_txt, 'a') as f:
    line_txt = [yolo_train_path + str(counter) + '.jpg', '\n']
    f.writelines(line_txt)
  counter += 1

In [ ]:
write_txt

'/content/drive/My Drive/project/mango/mango_detection/cfg/train.txt'